## 참고 링크
- 데이터 전처리 참고 : https://dacon.io/competitions/official/235747/codeshare/3054
- 데이터 다운로드 : https://www.bigkinds.or.kr/v2/news/index.do 
- 뉴스 감성지수 분류 모델 참고 : https://sieon-dev.tistory.com/15
- 기사 제목으로 긍정 부정 중립 분류 모델 만들기 참고 : https://somjang.tistory.com/entry/Keras%EA%B8%B0%EC%82%AC-%EC%A0%9C%EB%AA%A9%EC%9D%84-%EA%B0%80%EC%A7%80%EA%B3%A0-%EA%B8%8D%EC%A0%95-%EB%B6%80%EC%A0%95-%EC%A4%91%EB%A6%BD-%EB%B6%84%EB%A5%98%ED%95%98%EB%8A%94-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EC%96%B4%EB%B3%B4%EA%B8%B0
- 자연어 처리 참고 : https://wikidocs.net/book/2155

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings("ignore")
import hanja
from hanja import hangul
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os

## 1. 데이터 로드

In [3]:
# 작업경로 확인
print(os.getcwd())

c:\Workspace\HK_Project\7_자연어처리


In [ ]:
os.makedirs("c:/Workspace/HK_Project/7_자연어처리/data")

In [ ]:
# 파일 경로 설정
filePath = "c:/Workspace/HK_Project/7_자연어처리/data/bigkinds_NewsData.xlsx"

In [ ]:
# 디렉토리 변경 : `os.chdir("가고 싶은 경로")`

In [13]:
# 경로에서 파일명만 확인
os.path.basename(filePath)

'bigkinds_NewsData.xlsx'

In [5]:
df = pd.read_excel("data/bigkinds_NewsData.xlsx")
df.head(2)

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100201e+06,20220316,머니투데이,강상구|MTN PD,[갈 테마 가라] 2분기엔 턴어라운드 미리 선점해야 할 반도체 탑픽은?,경제>반도체,경제>유통,경제>자동차,NaN,NaN,NaN,NaN,신사,"머니투데이방송,MTNW,PMI,SK하이닉스","2분기,턴어라운드,선점,반도체,탑픽,진행,앵커,최희정,머니투데이방송,출연,어드바이저...","반도체,김진만,반도체주,최희정,머니투데이방송,구체화,턴어라운드,하이닉스,sic,성수...",[머니투데이 강상구 MTN PD] \n\n\n진행 : 최희정 머니투데이방송 앵커\n...,http://news.moneytoday.co.kr/view/mtview.php?n...,NaN
1,2.100601e+06,20220316,한국경제,강경주,"인텔, 작정하고 유럽에 100조 넘게 '반도체 투자'",경제>반도체,경제>자동차,경제>유통,NaN,NaN,NaN,"겔싱어,우르줄라","프랑스,미국,폴란드,고성,유럽,대유럽,바르셀로나엔,아시아,스페인,최고경영자,마그데부...","EU,인텔,라이엔,유럽연합(","인텔,작정,유럽,100조,반도체,투자,미국,반도체,기업,인텔,10년,유럽,반도체,생...","인텔,반도체,유로,eu,유럽,미국,폴란드,프랑스,우르줄라,파운드리,겔싱어,바르셀로나...",미국 반도체 기업 인텔이 앞으로 10년간 유럽에 반도체 생산과 연구 개발(R&D)을...,https://www.hankyung.com/economy/article/20220...,NaN


## 2. 데이터 확인

In [6]:
# 데이터 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2412 entries, 0 to 2411
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   뉴스 식별자             2412 non-null   float64
 1   일자                 2412 non-null   int64  
 2   언론사                2412 non-null   object 
 3   기고자                2367 non-null   object 
 4   제목                 2412 non-null   object 
 5   통합 분류1             2412 non-null   object 
 6   통합 분류2             2284 non-null   object 
 7   통합 분류3             1963 non-null   object 
 8   사건/사고 분류1          184 non-null    object 
 9   사건/사고 분류2          16 non-null     object 
 10  사건/사고 분류3          4 non-null      object 
 11  인물                 1453 non-null   object 
 12  위치                 2176 non-null   object 
 13  기관                 2403 non-null   object 
 14  키워드                2412 non-null   object 
 15  특성추출(가중치순 상위 50개)  2412 non-null   object 
 16  본문                 2412 

In [7]:
df.drop(columns=[
                "뉴스 식별자", "기고자", "통합 분류1", "통합 분류2", "통합 분류3", 
                "사건/사고 분류1", "사건/사고 분류2", "사건/사고 분류3", "인물", 
                "위치", "기관", "키워드", "특성추출(가중치순 상위 50개)", "본문",
                "URL", "분석제외 여부"], inplace=True)
df.head(2)

,일자,언론사,제목
0,20220316,머니투데이,[갈 테마 가라] 2분기엔 턴어라운드 미리 선점해야 할 반도체 탑픽은?
1,20220316,한국경제,"인텔, 작정하고 유럽에 100조 넘게 '반도체 투자'"


In [8]:
df.drop(columns=["언론사"], inplace=True)

In [9]:
# 일자, 제목 컬럼 확인
df.head(2)

,일자,제목
0,20220316,[갈 테마 가라] 2분기엔 턴어라운드 미리 선점해야 할 반도체 탑픽은?
1,20220316,"인텔, 작정하고 유럽에 100조 넘게 '반도체 투자'"


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2412 entries, 0 to 2411
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   일자      2412 non-null   int64 
 1   제목      2412 non-null   object
dtypes: int64(1), object(1)
memory usage: 37.8+ KB


In [11]:
# 결측치 확인
df.isnull().sum()

일자    0
제목    0
dtype: int64

## 3. train, test dataset 생성

In [12]:
# train, test set 분리
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df)

In [13]:
# train_data 확인
train_data.head(2)

,일자,제목
73,20220314,"""유동성 줄어도 시장은 상승 리오프닝 반도체 주목""[2022 한경스타워즈 출사표③]"
562,20220220,"인텔, 車 반도체 파운드리 진출 글로벌 경쟁 격화"


In [14]:
# 데이터 개수 : 1809
# 컬럼 수 : 2
train_data.shape

(1809, 2)

In [15]:
# test_data 확인
test_data.head(2)

,일자,제목
261,20220303,"“D램 초격차 재입증” 삼성전자, 차세대 저전력 반도체 검증 성공"
660,20220216,반도체 투자 활성화를 위한 기업 간담회 개최


In [16]:
# 데이터 개수 : 603
# 컬럼 수 : 2
test_data.shape

(603, 2)

## 4. 한글, 영어, 숫자 이외 데이터 제거

In [25]:
train_data["제목"] = train_data["제목"].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣|A-Za-z|0-9 ]', '')

In [26]:
train_data.isnull().sum()

일자    0
제목    0
dtype: int64

## 5. 한자가 들어간 데이터 확인

In [17]:
from collections import Counter

In [18]:
len(train_data)

1809

In [19]:
train_hanja_data = train_data.copy()

In [20]:
pattern = re.compile('[一-龥]')

In [21]:
# 한자 리스트 확인
hanja_list = train_hanja_data["제목"].str.findall(pattern).sum()
hanja_list[:5]

['車', '日', '美', '韓', '車']

In [22]:
# 빈도수가 높은 한자 체크
Counter(hanja_list).most_common()[:5]

[('美', 132), ('中', 107), ('車', 106), ('株', 48), ('日', 47)]

In [23]:
# 한자 중복값 제거 확인
new_list = []
for i in hanja_list:
    if i not in new_list:
        new_list.append(i)
print(new_list)

['車', '日', '美', '韓', '文', '印', '中', '株', '安', '稅', '企', '亞', '前', '亂', '年', '兆', '獨', '國', '史', '發', '比', '社', '尹', '論', '李', '野', '全', '外', '人', '弗', '乙', '好', '向', '對', '市', '夢', '新', '産', '證', '號', '法', '戰', '說', '價', '硏']


In [106]:
d = ['株', '中', '伊', '韓', '社', 
'對', '美', '日', '車', '獨', 
'文', '印', '價', '安', '外', 
'人', '亂', '産', '兆', '全', 
'企', '市', '國', '亞', '尹', 
'弗', '發', '史', '證', '夢', 
'論', '檢', '好', '說', '大', 
'年', '乙', '前', '稅', '李', 
'法', '新', '比', '硏', '戰', 
'號', '野']

kor = ["주식", "중국", "이탈리아", "한국", "기업", 
"상대", "미국", "일본", "자동차", "독일",
"문", "인도", "값", "안", "이외",
"인", "난리", "생산", "조 원", "전체",
"기업", "시", "국가", "아시아", "윤",
"달러", "발", "역사", "증권", "몽",
"론", "검찰", "호", "설", "대",
"연", "을", "전", "세금", "이",
"법", "새", "대비", "연구원", "전쟁",
"호", "야당"]

In [108]:
dictionary = dict(zip(d, kor))
print(dictionary)

{'株': '주식', '中': '중국', '伊': '이탈리아', '韓': '한국', '社': '기업', '對': '상대', '美': '미국', '日': '일본', '車': '자동차', '獨': '독일', '文': '문', '印': '인도', '價': '값', '安': '안', '外': '이외', '人': '인', '亂': '난리', '産': '생산', '兆': '조 원', '全': '전체', '企': '기업', '市': '시', '國': '국가', '亞': '아시아', '尹': '윤', '弗': '달러', '發': '발', '史': '역사', '證': '증권', '夢': '몽', '論': '론', '檢': '검찰', '好': '호', '說': '설', '大': '대', '年': '연', '乙': '을', '前': '전', '稅': '세금', '李': '이', '法': '법', '新': '새', '比': '대비', '硏': '연구원', '戰': '전쟁', '號': '호', '野': '야당'}


- pandas와 dictionary matching 후, 변환 참고: https://stackoverflow.com/questions/31491727/how-to-match-dictionary-values-to-pandas-data-frame-values

## 6. 한자 -> 한글로 대체
    - 뉴스 제목에 사용되는 한자들은 의미를 축약한 한 글자로 구성된 한자가 많은 것으로 추측됨
    - 한자 -> 한글로 단순 번역하는 방법을 시도해보기로 계획

In [28]:
list(map(lambda x: [ hanja.translate(line, "substitution") for line in train_data["제목"] ], train_data))[0]
train_data.head(3)

,일자,제목
73,20220314,유동성 줄어도 시장은 상승 리오프닝 반도체 주목2022 한경스타워즈 출사표
562,20220220,인텔 반도체 파운드리 진출 글로벌 경쟁 격화
795,20220211,반도체전시회 참석한 현대모비스 조성환 사장 미래차 핵심은 반도체 소프트웨어 기술력


## 7. test_data : train_data와 동일한 방식으로 진행

In [38]:
# test_data의 shape 확인
test_data.shape

(603, 2)

In [39]:
# 한글, 영어, 숫자 이외 데이터 제거
test_data["제목"] = test_data["제목"].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣|A-Za-z|0-9 ]', '')
# 결측치 확인
test_data.isnull().sum()

일자    0
제목    0
dtype: int64

In [44]:
# 한자 -> 한글로 대체
list(map(lambda x: [ hanja.translate(line, "substitution") for line in test_data["제목"] ], test_data))[0]
test_data.head(3)

,일자,제목
261,20220303,D램 초격차 재입증 삼성전자 차세대 저전력 반도체 검증 성공
660,20220216,반도체 투자 활성화를 위한 기업 간담회 개최
781,20220213,반도체 업계 기술에는 정년 없다 전문가 붙잡는 삼성전자 SK하이닉스


## 8. 명사만 추출한 컬럼 추가
    - 참고 : https://sieon-dev.tistory.com/15

### 8-1. train_data

In [50]:
okt = Okt()
n_ = []
title_rename = []
for i in range(len(train_data)):
    if (i % 1809 == 0):
        print(i, "단계완료")
    n_.append(' '.join(okt.nouns(train_data.iloc[i]["제목"])))


# 명사 추출 컬럼 
train_data["nouns"] = n_

# empty space가 아닌 nouns 컬럼만 train_data에 저장
train_data = train_data[train_data["nouns"] != ''] 

0 단계완료


In [51]:
train_data.head(3)

,일자,제목,nouns
73,20220314,유동성 줄어도 시장은 상승 리오프닝 반도체 주목2022 한경스타워즈 출사표,유동성 시장 상승 리 오프닝 반도체 주목 스타워즈 출사표
562,20220220,인텔 반도체 파운드리 진출 글로벌 경쟁 격화,인텔 반도체 파운드리 진출 글로벌 경쟁 격화
795,20220211,반도체전시회 참석한 현대모비스 조성환 사장 미래차 핵심은 반도체 소프트웨어 기술력,반도체 전시회 참석 현대 모비스 조성환 사장 미래 차 핵심 반도체 소프트웨어 기술


### 8-2. test_data

In [52]:
okt = Okt()
n_ = []
title_rename = []
for i in range(len(test_data)):
    if (i % 603 == 0):
        print(i, "단계완료")
    n_.append(' '.join(okt.nouns(test_data.iloc[i]["제목"])))


# 명사 추출 컬럼 
test_data["nouns"] = n_

# empty space가 아닌 nouns 컬럼만 test_data에 저장
test_data = test_data[test_data["nouns"] != ''] 

0 단계완료


In [53]:
test_data.head(3)

,일자,제목,nouns
261,20220303,D램 초격차 재입증 삼성전자 차세대 저전력 반도체 검증 성공,램 초 격차 재 입증 삼성 전자 차세대 전력 반도체 검증 성공
660,20220216,반도체 투자 활성화를 위한 기업 간담회 개최,반도체 투자 활성화 위 기업 간담 회 개최
781,20220213,반도체 업계 기술에는 정년 없다 전문가 붙잡는 삼성전자 SK하이닉스,반도체 업계 기술 정년 전문가 삼성 전자 하이닉스


- 에러내용 : JVMNotFoundException: No JVM shared library file (jvm.dll) found.
    - 참고 : https://stricky.tistory.com/398

In [46]:
# python bit 확인
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


- 오라클 JDK 다운로드 사이트 :  https://www.oracle.com/java/technologies/downloads/#java8

- 설치 경로 확인 > 시스템 환경 변수에 추가 > 변수이름 `JAVA_HOME`으로 저장

# ★ 2022-03-17 11:29 이후 Plan
## `[데이터 전처리]`
1. 반도체 기업 13개 일자별 상승 및 하락 데이터 & 뉴스 제목 명사 추출 
    - 전일 대비 상승 : updown 1 태깅
    - 전일 대비 하락 : updown 0 태깅
2. `1`과 `0`의 비율 확인
3. 감성사전 구축 : 한 글자인 단어는 제거, 두 글자 이상인 단어들의 점수 초기화
4. sent_score 컬럼 추가
    - 뉴스의 평균 감성 점수 산출
5. sent_label 추가
    - sent_scroe > sent_label : 1 
    - sent_scroe < sent_label : 0

## `[모델링]`
1. 형태소 분류기 설치
    - Mecab
2. 모델 사용 방안
    - Bi-LSTM
    - BERT
3. X, Y 값 지정
    - train_set, test_set
    - X, Y 값 지정
4. 벡터화
    - keras의 Tokenizer 사용
5. 일정한 Input 형태를 만들기 위해 길이 지정, padding 진행
6. 모델 구현
7. 성능 확인